In [4]:

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler, PolynomialFeatures
# from sklearn.linear_model import Ridge
# from sklearn.metrics import mean_squared_error, r2_score

# # 1. Load the dataset
# file_path = 'combined_file.csv'
# df = pd.read_csv(file_path)

# # 2. Robust Date Parsing
# df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# # 3. Feature Engineering (Time Series Lags)
# df = df.sort_values('Date')

# # Create robust history features using temporary forward-filling
# df_filled = df.copy()
# df_filled['Recommended Feed to Hydrolyser'] = df_filled['Recommended Feed to Hydrolyser'].ffill()
# df_filled['Recommonded Feed to Digester'] = df_filled['Recommonded Feed to Digester'].ffill()

# # Create Lags (1, 2, 3 days history)
# for lag in [1, 2, 3]:
#     df[f'Hydro_Lag{lag}'] = df_filled['Recommended Feed to Hydrolyser'].shift(lag)
#     df[f'Digester_Lag{lag}'] = df_filled['Recommonded Feed to Digester'].shift(lag)

# # Create Rolling Means (3-day average)
# df['Hydro_RollMean3'] = df_filled['Recommended Feed to Hydrolyser'].shift(1).rolling(window=3).mean()
# df['Digester_RollMean3'] = df_filled['Recommonded Feed to Digester'].shift(1).rolling(window=3).mean()

# # Date Components
# df['Month'] = df['Date'].dt.month
# df['Day'] = df['Date'].dt.day
# df['DayOfWeek'] = df['Date'].dt.dayofweek

# # 4. Define Features and Targets
# X_cols = [
#     'Initial Hydrolyser pH',
#     'Initial Digester pH',
#     'No of HCl drops added to convert Digester pH to 5.0pH',
#     'No of HCl drops added to conver 5.0pH to 4.4pH',
#     'No of HCl drops added to conver 4.4pH to 4.0pH',
#     'No of HCl drops added to conver Digester pH to 4.0pH',
#     # Time Series Features
#     'Hydro_Lag1', 'Hydro_Lag2', 'Hydro_Lag3',
#     'Digester_Lag1', 'Digester_Lag2', 'Digester_Lag3',
#     'Hydro_RollMean3', 'Digester_RollMean3',
#     # Date Features
#     'Month', 'Day', 'DayOfWeek'
# ]

# y_cols = [
#     'Recommended Feed to Hydrolyser',
#     'Recommonded Feed to Digester'
# ]

# # 5. Data Cleaning
# # Drop rows with any missing values in the selected columns
# df_clean = df.dropna(subset=X_cols + y_cols).copy()

# X = df_clean[X_cols]
# y = df_clean[y_cols]

# print(f"Training on {len(X)} samples.")

# # 6. Split Data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # 7. Scale Features
# # Scaling is very important before applying Polynomial transformations
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# # 8. Polynomial Transformation
# # degree=2 creates interactions (x1*x2, x1^2, etc.)
# poly = PolynomialFeatures(degree=2, include_bias=False)
# X_train_poly = poly.fit_transform(X_train_scaled)
# X_test_poly = poly.transform(X_test_scaled)

# print(f"Number of features after Polynomial expansion: {X_train_poly.shape[1]}")

# # 9. Train Polynomial Regression Model (using Ridge)
# # Ridge is used instead of simple LinearRegression to control overfitting on the 170+ features
# poly_model = Ridge(alpha=1.0)
# poly_model.fit(X_train_poly, y_train)

# # 10. Predict and Evaluate
# y_pred = poly_model.predict(X_test_poly)

# r2 = r2_score(y_test, y_pred)
# mse = mean_squared_error(y_test, y_pred)

# print("-" * 30)
# print(f"Polynomial Regression Results:")
# print(f"R2 Score: {r2:.4f}")
# print(f"Mean Squared Error: {mse:.2f}")
# print("-" * 30)

# # 11. Visualize
# results = pd.DataFrame(y_test, columns=['Actual Hydro', 'Actual Digester'])
# results['Pred Hydro'] = y_pred[:, 0]
# results['Pred Digester'] = y_pred[:, 1]

# plt.figure(figsize=(12, 5))
# plt.subplot(1, 2, 1)
# plt.scatter(range(len(results)), results['Actual Hydro'], label='Actual', color='blue', alpha=0.6)
# plt.scatter(range(len(results)), results['Pred Hydro'], label='Predicted', marker='x', color='red', alpha=0.6)
# plt.title('Polynomial Reg: Hydrolyser Feed')
# plt.legend()

# plt.subplot(1, 2, 2)
# plt.scatter(range(len(results)), results['Actual Digester'], label='Actual', color='green', alpha=0.6)
# plt.scatter(range(len(results)), results['Pred Digester'], label='Predicted', marker='x', color='orange', alpha=0.6)
# plt.title('Polynomial Reg: Digester Feed')
# plt.legend()

# plt.tight_layout()
# plt.savefig('poly_regression_results.png')
# print("Plot saved.")


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, r2_score

# Try to import SHAP (for Explainable AI)
try:
    import shap
    SHAP_AVAILABLE = True
except ImportError:
    SHAP_AVAILABLE = False
    print("Warning: 'shap' library not found. Install it using 'pip install shap' to see explanations.")

# ==========================================
# 1. Load and Preprocess Data
# ==========================================
file_path = 'combined_file.csv'
df = pd.read_csv(file_path)

# Robust Date Parsing
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.sort_values('Date')

# Feature Engineering: Create History (Lag) Features
# We use a temporary dataframe with forward-fill to calculate previous day's values robustly
df_filled = df.copy()
df_filled['Recommended Feed to Hydrolyser'] = df_filled['Recommended Feed to Hydrolyser'].ffill()
df_filled['Recommonded Feed to Digester'] = df_filled['Recommonded Feed to Digester'].ffill()

# Create Lags (1, 2, 3 days back)
for lag in [1, 2, 3]:
    df[f'Hydro_Lag{lag}'] = df_filled['Recommended Feed to Hydrolyser'].shift(lag)
    df[f'Digester_Lag{lag}'] = df_filled['Recommonded Feed to Digester'].shift(lag)

# Create Rolling Means (3-day average)
df['Hydro_RollMean3'] = df_filled['Recommended Feed to Hydrolyser'].shift(1).rolling(window=3).mean()
df['Digester_RollMean3'] = df_filled['Recommonded Feed to Digester'].shift(1).rolling(window=3).mean()

# Extract Date Features
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['DayOfWeek'] = df['Date'].dt.dayofweek

# Define Features (X) and Targets (y)
X_cols = [
    'Initial Hydrolyser pH', 'Initial Digester pH',
    'No of HCl drops added to convert Digester pH to 5.0pH',
    'No of HCl drops added to conver 5.0pH to 4.4pH',
    'No of HCl drops added to conver 4.4pH to 4.0pH',
    'No of HCl drops added to conver Digester pH to 4.0pH',
    'Hydro_Lag1', 'Hydro_Lag2', 'Hydro_Lag3',
    'Digester_Lag1', 'Digester_Lag2', 'Digester_Lag3',
    'Hydro_RollMean3', 'Digester_RollMean3',
    'Month', 'Day', 'DayOfWeek'
]

y_cols = ['Recommended Feed to Hydrolyser', 'Recommonded Feed to Digester']

# Data Cleaning: Drop rows where any feature or target is missing
df_clean = df.dropna(subset=X_cols + y_cols).copy()

X = df_clean[X_cols]
y = df_clean[y_cols]

print(f"Data successfully processed. Training on {len(X)} samples.")

# Split Data (80% Train, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ==========================================
# 2. Train Polynomial Regression Model
# ==========================================
# We use a Pipeline: Scale -> Polynomial Transform -> Ridge Regression
model_pipeline = make_pipeline(
    StandardScaler(),
    PolynomialFeatures(degree=2, include_bias=False),
    Ridge(alpha=1.0)
)

model_pipeline.fit(X_train, y_train)

# ==========================================
# 3. Evaluate and Show Results
# ==========================================
y_pred = model_pipeline.predict(X_test)

r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("\n" + "="*40)
print(f"MODEL PERFORMANCE")
print("="*40)
print(f"R2 Score: {r2:.4f}")
print(f"Mean Squared Error: {mse:.2f}")

# --- DISPLAY RESULTS TABLE ---
# Create a readable DataFrame comparing Actual vs Predicted values
results_df = pd.DataFrame(
    y_test.values,
    columns=['Actual Hydro', 'Actual Digester'],
    index=y_test.index
)
results_df['Pred Hydro'] = y_pred[:, 0]
results_df['Pred Digester'] = y_pred[:, 1]

# Calculate difference (Error)
results_df['Diff Hydro'] = results_df['Actual Hydro'] - results_df['Pred Hydro']

print("\n" + "="*40)
print("SAMPLE PREDICTIONS (ACTUAL vs PREDICTED)")
print("="*40)
print(results_df.head(10))  # Show first 10 predictions
print("="*40)

# ==========================================
# 4. Explainable AI (SHAP)
# ==========================================
if SHAP_AVAILABLE:
    print("\nGenerating SHAP Explanations...")

    # 1. Summarize background data (X_train) to speed up computation
    # KernelExplainer is computationally expensive, so we use k-means to find 10 representative samples
    X_train_summary = shap.kmeans(X_train, 10)

    # 2. Define a prediction wrapper
    # SHAP needs a function that takes X and returns ONE target prediction at a time.
    # Here we explain Target 0: 'Recommended Feed to Hydrolyser'
    def predict_hydrolyser(data):
        return model_pipeline.predict(data)[:, 0]

    # 3. Initialize Explainer
    explainer = shap.KernelExplainer(predict_hydrolyser, X_train_summary)

    # 4. Calculate SHAP values for the test set
    print("Calculating SHAP values (this may take a minute)...")
    shap_values = explainer.shap_values(X_test)

    # 5. Plot: Summary Plot (Global Importance)
    plt.figure(figsize=(10, 6))
    plt.title("Feature Importance (SHAP) - Hydrolyser Feed")
    shap.summary_plot(shap_values, X_test, show=False)
    plt.tight_layout()
    plt.savefig('shap_summary_hydrolyser.png')
    print(">> Saved plot: 'shap_summary_hydrolyser.png'")

    # 6. Plot: Bar Plot (Simpler View)
    plt.figure(figsize=(10, 6))
    plt.title("Feature Importance Bar Plot - Hydrolyser Feed")
    shap.summary_plot(shap_values, X_test, plot_type="bar", show=False)
    plt.tight_layout()
    plt.savefig('shap_barplot_hydrolyser.png')
    print(">> Saved plot: 'shap_barplot_hydrolyser.png'")

    print("SHAP analysis complete.")
else:
    print("\nSkipping SHAP analysis (library not installed).")

Data successfully processed. Training on 94 samples.

MODEL PERFORMANCE
R2 Score: 0.7553
Mean Squared Error: 2758.82

SAMPLE PREDICTIONS (ACTUAL vs PREDICTED)
     Actual Hydro  Actual Digester  Pred Hydro  Pred Digester  Diff Hydro
58          400.0            400.0  415.710127     420.283626  -15.710127
33          350.0            350.0  355.044130     360.857135   -5.044130
82          400.0            400.0  371.507004     422.938174   28.492996
140         400.0            400.0  372.255544     394.289141   27.744456
2           100.0            150.0  256.061780     191.514590 -156.061780
37          350.0            350.0  334.403210     349.373317   15.596790
57          400.0            400.0  170.613368     406.371133  229.386632
132         400.0            400.0  406.518872     399.807257   -6.518872
16          250.0            250.0  297.204517     265.744639  -47.204517
64          400.0            400.0  402.052530     396.819776   -2.052530

Generating SHAP Explanatio

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


  0%|          | 0/19 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/ut

>> Saved plot: 'shap_summary_hydrolyser.png'
>> Saved plot: 'shap_barplot_hydrolyser.png'
SHAP analysis complete.
